**Author:** Revekka Gersgovich

**Purpose:** Explore the data with children's mental health stigma or adult mental health stigma modules

**Date:** Nov 29, 2025

In [ ]:
import os
import os.path as path
import pandas as pd
import numpy as np
import glob
import narwhals
import pyreadstat
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import geopandas as gpd
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
parent_dir = os.path.abspath("/Users/revekkagershovich/Documents/Filling_system/Academic/Taste-Based_Discrimination") # Change this directory to run from your computer
assert os.path.exists(parent_dir), "parent_dir does not exist"
os.chdir(parent_dir)

raw_data_dir = path.join(parent_dir, "1_data", "1_raw")
assert os.path.exists(raw_data_dir), "raw_data_dir does not exist"

intermediate_data_dir = path.join(parent_dir, "1_data", "2_intermediate")
assert os.path.exists(intermediate_data_dir), "intermediate_data_dir does not exist"

exhibits_dir = path.join(parent_dir, "3_exhibits")
assert os.path.exists(exhibits_dir), "exhibits_dir does not exist"

# Loading Data

In [ ]:
df = pd.read_csv(path.join(intermediate_data_dir, "gss_cleaned_96_02_06_18_24.csv"))

In [ ]:
# States shapefile with regions
shp_path = os.path.join(raw_data_dir, "s_05mr24", "s_05mr24.shp")
states = gpd.read_file(shp_path)

state_to_region = {
    # 1 = Northeast
    'ME':1, 'NH':1, 'VT':1, 'MA':1, 'RI':1, 'CT':1, 'NY':1, 'NJ':1, 'PA':1,
    # 2 = Midwest
    'OH':2, 'IN':2, 'IL':2, 'MI':2, 'WI':2, 'MN':2, 'IA':2, 'MO':2, 'ND':2, 'SD':2, 'NE':2, 'KS':2,
    # 3 = South
    'DE':3, 'MD':3, 'DC':3, 'VA':3, 'WV':3, 'NC':3, 'SC':3, 'GA':3, 'FL':3,
    'KY':3, 'TN':3, 'AL':3, 'MS':3, 'AR':3, 'LA':3, 'OK':3, 'TX':3,
    # 4 = West
    'MT':4, 'ID':4, 'WY':4, 'CO':4, 'NM':4, 'AZ':4, 'UT':4, 'NV':4,
    'WA':4, 'OR':4, 'CA':4, 'AK':4, 'HI':4
}

states['region'] = states['STATE'].map(state_to_region)

In [ ]:
df.shape

# Filtering Data

In [ ]:
df['upsdowns'].value_counts()

In [ ]:
# Dropping respondents who did not receive either childhood mental health stigma module or adult mental health stigma module
df["vignette_missing"] = (
    df["vigversn"].isna()
).astype(int)

print(df['vignette_missing'].value_counts())

In [ ]:
# Dropping respondents who did not receive either childhood mental health stigma module or adult mental health stigma module
df["vignette_missing"] = (
    df["vigversn"].isna() & df["chldvig"].isna()
).astype(int)

print(df['vignette_missing'].value_counts())

In [ ]:
# 2. Calculate share missing by year
miss = df.groupby("year")["vignette_missing"].mean()

# 3. Plot
miss.plot(kind="bar")
plt.ylabel("Share missing")
plt.title("Share of respondents with vignette_missing by year")
plt.ylim(0, 1)
plt.show()

In [ ]:

df = df[df['vignette_missing'] == 0].copy()
df.drop(columns=['vignette_missing'], inplace=True)
print(df.shape)

In [ ]:
df['year'].value_counts()

In [ ]:
mh_vars = [
    "evbrkdwn",   # ever felt like having a nervous breakdown
    "relmhsp1",   # patient was self (mental health help-seeking)
    "evmhp",      # ever had a mental health problem
    "mntlhlth",   # days of poor mental health, past 30 days
    "depress",    # ever told by a doctor you had depression
    "diagnosd",   # ever diagnosed with mental health problem
    "mhtreatd",   # ever treated for mental health problem
    "emoprobs"    # emotional problems interfering with life
]

In [ ]:
df["mh_all_missing"] = df[mh_vars].isna().all(axis=1).astype(int)

print(df['mh_all_missing'].value_counts())

In [ ]:
df.shape

In [ ]:
df_full = df.copy()

In [ ]:
# Keep only respondents whose ADULT vignette is a "problem" vignette (vigversn < 73)
df = df[(df['vigversn'].isna()) | (df['vigversn'] < 73.0)].copy()

# Keep only respondents whose CHILD vignette is NOT in 17–24 (those are the "no problem" child vignettes)
df = df[(df['chldvig'].isna()) | ((df['chldvig'] < 17.0) | (df['chldvig'] > 24.0))].copy()

In [ ]:
df['vigversn'].dtype

In [ ]:
df.shape

# Creating a unified measure of respondents' mental health

## Converting continuous variables to dummies

In [ ]:
# df["mntlhlth_dummy_cdc"] = (df["mntlhlth"] >= 14).astype(int)

# df["mntlhlth_dummy_cdc"] = df["mntlhlth_dummy_cdc"].replace({0: 2})

df["mntlhlth_dummy"] = (df["mntlhlth"] >= 4).astype(int)

df["mntlhlth_dummy"] = df["mntlhlth_dummy"].replace({0: 2})

df[~df['mntlhlth'].isna()][['mntlhlth', 'mntlhlth_dummy']].head()

# Part of CDC healthy days framework



In [ ]:
# assuming the responses are coded 1–5
df["emoprobs_dummy"] = df["emoprobs"].isin([4, 5]).astype(int)

df["emoprobs_dummy"] = df["emoprobs_dummy"].replace({0: 2})

# Often and Always are coded as 1

df[~df['emoprobs'].isna()][['emoprobs', 'emoprobs_dummy']].head()

## Exploring which own respondent mental health measures are available for which years

In [ ]:
mh_vars = [
    "evbrkdwn",   # ever felt like having a nervous breakdown - 1 "yes", 2 "no"
    "relmhsp1",   # patient was self (mental health help-seeking) - 1 "self", 2 "no"
    "evmhp",      # ever had a mental health problem - 1 "yes", 2 "no"
    "mntlhlth_dummy",   # days of poor mental health, past 30 days more than 14 - 1 "yes", 2 "no"
    "depress",    # ever told by a doctor you had depression - 1 "yes", 2 "no"
    "diagnosd",   # ever diagnosed with mental health problem - 1 "yes", 2 "no"
    "mhtreatd",   # ever treated for mental health problem - 1 "yes", 2 "no"
    "emoprobs_dummy"    # emotional problems interfering with life - 1 "yes", 2 "no"
]

In [ ]:
years = sorted(df['year'].unique())
rows = []

for var in mh_vars:
    row = {"variable": var}
    for yr in years:
        # Check if the variable has any non-missing values for that year
        present = df.loc[df['year'] == yr, var].eq(1).any()
        row[str(yr)] = 1 if present else 0
    rows.append(row)

mh_presence = pd.DataFrame(rows)
print(mh_presence)

In [ ]:
mh_vars_2018 = [
    "mntlhlth_dummy",
    "diagnosd",
    "mhtreatd"
]

# 1. Filter 2018 only
df18 = df[df["year"] == 2018]

# 2. Compute % == 1 for each variable
mh_stats = (
    df18[mh_vars_2018]
    .eq(1)
    .mean()
    .mul(100)
    .sort_values()
)

print(mh_stats)

# 3. Plot
plt.figure(figsize=(8, 4))
plt.barh(mh_stats.index, mh_stats.values)
plt.xlabel("Percent of respondents with indicator = 1")
plt.title("Prevalence of Mental-Health Measures in 2018")
plt.tight_layout()
plt.show()

## Plotting simple graph of all mental health measures over the years

In [ ]:
# # Convert the mh_vars to numeric just to be safe
df[mh_vars] = df[mh_vars].apply(pd.to_numeric, errors='coerce')

# Dummy = 1 if any variable == 1
df["mh_any_problem"] = df[mh_vars].eq(1).any(axis=1).map({True: 1, False: 2})

In [ ]:
mh_by_year = (
    df.groupby("year")["mh_any_problem"]
      .apply(lambda s: (s == 1).mean() * 100)   # percentage
      .reset_index(name="pct_mh_any_problem")
)

print(mh_by_year)

# 2. Plot
plt.figure()
plt.bar(mh_by_year["year"].astype(str), mh_by_year["pct_mh_any_problem"])
plt.xlabel("Year")
plt.ylabel("Percent with any mental health problem (mh_any_problem == 1)")
plt.title("Share of respondents with any mental health problem, by year")
plt.tight_layout()
plt.show()

# Creating Synthetic Stigma Measure

In [ ]:
# Recode stigma variables to be binary: 1 = unwilling (3,4 - more stigma), 0 = willing (1,2 - less stigma), None = missing
stigma_vars = [
    'vigfrnd','viggrp','vigmar','vignei','vigsoc','vigwork',
    'chldfrnd','chldsch','nextdoor','spendeve'
]

for v in stigma_vars:
    df[v] = df[v].apply(lambda x: 1 if x in [3,4] else (0 if x in [1,2] else None))
    
df['vigfrnd'].value_counts()

In [ ]:
# create standardized versions: var_z
for v in stigma_vars:
    df[v + '_z'] = (
        df.groupby('year')[v]
          .transform(lambda x: (x - x.mean()) / x.std())
    )

In [ ]:
z_cols = [v + '_z' for v in stigma_vars]

df['stigma_index'] = df[z_cols].mean(axis=1, skipna=True)

In [ ]:
stigma_by_region_year = (
    df
    # e.g. restrict to non-MH respondents if you want:
    # .loc[df['MH'] == 0]
    .groupby(['region', 'year'])['stigma_index']
    .mean()
    .reset_index()
)

In [ ]:
# 1. Map region codes to names
region_map = {
    1: 'Northeast',
    2: 'Midwest',
    3: 'South',
    4: 'West'
}

tbl = stigma_by_region_year.copy()
tbl['Region'] = tbl['region'].map(region_map)

# 2. Pivot: rows = Region, cols = Year
tbl_wide = (
    tbl.pivot(index='Region', columns='year', values='stigma_index')
       .reindex(['Northeast', 'Midwest', 'South', 'West'])  # order rows
       .round(3)
)

# 3. Optional: nicer column name format (e.g., int)
tbl_wide.columns = tbl_wide.columns.astype(int)

tbl_wide

In [ ]:
# 4. Convert to LaTeX
latex_str = tbl_wide.to_latex(
    caption='Mean Stigma Index by Region and Year',
    label='tab:stigma_region_year',
    na_rep='',
    column_format='l' + 'r'*len(tbl_wide.columns)
)

# 5. Save to file
out_path = os.path.join(exhibits_dir, 'stigma_region_year.tex')
with open(out_path, 'w') as f:
    f.write(latex_str)

print("Saved LaTeX table to:", out_path)

# Filtering to two datasets

## Data for 2002, 2006, and 2018

In [ ]:
df02_06_18 = df[df['year'].isin([2002, 2006, 2018])].copy()
df02_06_18.shape

In [ ]:
df02_06_18.drop(columns=['evbrkdwn', 'relmhsp1', 'evmhp', 'depress', 'diagnosd', 'mhtreatd','emoprobs', 'emoprobs_dummy'], inplace=True)

In [ ]:
df02_06_18[~df02_06_18['mntlhlth_dummy'].isna()]
print(df02_06_18.shape)

## Data for 2018, and 2024

In [ ]:
df18_24 = df[df['year'].isin([2018, 2024])].copy()
df18_24.shape

In [ ]:
df18_24.drop(columns=['evbrkdwn', 'relmhsp1', 'evmhp', 'depress', 'mntlhlth', 'mntlhlth_dummy', 'mhtreatd', 'emoprobs_dummy'], inplace=True)

In [ ]:
df18_24[~df18_24['diagnosd'].isna()]
print(df18_24.shape)

# Graph of Prevalence of Mental Health Disorders by Region and Year

In [ ]:
df02_06_18['mntlhlth_dummy'] = df02_06_18['mntlhlth_dummy'].replace({2:0, 1:1})
df02_06_18['mntlhlth_dummy'] = df02_06_18['mntlhlth_dummy'].astype(float)

In [ ]:
# Group by region and year, compute mean of the dummy = prevalence
prev = (
    df02_06_18
    .groupby(['region', 'year'])['mntlhlth_dummy']
    .mean()
    .reset_index()
)

# Convert to pivot table for easier plotting
table = prev.pivot(index='region', columns='year', values='mntlhlth_dummy')

# Sort columns for consistent order
table = table[[2002, 2006, 2018]]

In [ ]:
# Prevalence of Mental-Health Problems by Year and Region (clean/minimal + both axis lines)

fig, ax = plt.subplots(figsize=(8,5))

years = table.columns          # 2002, 2006, 2018
regions = table.index          # 1,2,3,4
x = np.arange(len(years))
width = 0.20

# Minimal theme: keep LEFT and BOTTOM axis lines, remove TOP and RIGHT
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(True)
ax.spines['bottom'].set_visible(True)

# Plot bars
for i, reg in enumerate(regions):
    ax.bar(
        x + i*width,
        table.loc[reg] * 100,
        width,
        label={1:'Northeast', 2:'Midwest', 3:'South', 4:'West'}[reg],
        alpha=0.9
    )

# Labels
ax.set_xlabel('Year', fontsize=11)
ax.set_ylabel('Prevalence (%)', fontsize=11)
ax.set_title('Mental-Health Problem Prevalence by Year and Region', fontsize=13)

# X-axis ticks
ax.set_xticks(x + width*1.5)
ax.set_xticklabels(years)

# No gridlines
ax.grid(False)

# Legend
ax.legend(title='Region', frameon=False)

plt.tight_layout()
plt.show()

out_path = os.path.join(exhibits_dir, "mh_prevalence_region02_06_18.png")
fig.savefig(out_path, dpi=300, bbox_inches='tight')

print("Saved figure to:", out_path)

In [ ]:
df18_24['diagnosd'] = df18_24['diagnosd'].replace({2:0})

# Group by region and year, compute mean of the dummy = prevalence
prev = (
    df18_24
    .groupby(['region', 'year'])['diagnosd']
    .mean()
    .reset_index()
)

# Convert to pivot table for easier plotting
table = prev.pivot(index='region', columns='year', values='diagnosd')

# Sort columns for consistent order
table = table[[2018, 2024]]

In [ ]:
# Prevalence of Mental-Health Problems by Year and Region (clean/minimal + both axis lines)

fig, ax = plt.subplots(figsize=(8,5))

years = table.columns          # 2002, 2006, 2018
regions = table.index          # 1,2,3,4
x = np.arange(len(years))
width = 0.20

# Minimal theme: keep LEFT and BOTTOM axis lines, remove TOP and RIGHT
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(True)
ax.spines['bottom'].set_visible(True)

# Plot bars
for i, reg in enumerate(regions):
    ax.bar(
        x + i*width,
        table.loc[reg] * 100,
        width,
        label={1:'Northeast', 2:'Midwest', 3:'South', 4:'West'}[reg],
        alpha=0.9
    )

# Labels
ax.set_xlabel('Year', fontsize=11)
ax.set_ylabel('Prevalence (%)', fontsize=11)
ax.set_title('Mental-Health Problem Prevalence by Year and Region', fontsize=13)

# X-axis ticks
ax.set_xticks(x + width*1.5)
ax.set_xticklabels(years)

# No gridlines
ax.grid(False)

# Legend – move it outside the plot, upper right
ax.legend(
    title='Region',
    frameon=False,
    loc='upper left'
)

plt.tight_layout()
plt.show()

out_path = os.path.join(exhibits_dir, "mh_prevalence_region18_24.png")
fig.savefig(out_path, dpi=300, bbox_inches='tight')

print("Saved figure to:", out_path)

# Preliminary Regression to show that my stigma variable behaves as it should. 

In [ ]:
# 0. Child vs adult vignette
df['vig_child'] = df['chldvig'].notna().astype(int)  # 1 = child vignette, 0 = adult


# 1. FILTER: drop non–mental-health child vignettes
# Keep child mental codes 14–16 (depression) and 33–56 (classic dep / impulsivity / perfectionism)
mask_child_mh = df['chldvig'].between(14, 16) | df['chldvig'].between(33, 56)
df.loc[df['vig_child'] == 1 & ~mask_child_mh, ['chldvig']] = np.nan  # knock out asthma/normal/etc.

# ---------- ADULT VIGNETTES: VIGVERSN 1–72 ----------

adult_cond_map = {
    0: 'alcohol_dependence',     # 1–18
    1: 'major_depression',       # 19–36
    2: 'schizophrenia',          # 37–54
    3: 'drug_problem'            # 55–72
}
adult_race_map = {0: 'white', 1: 'black', 2: 'hispanic'}
adult_educ_map = {0: '8th_grade', 1: 'high_school', 2: 'college'}

def decode_adult_vignette(code):
    if pd.isna(code) or not (1 <= int(code) <= 72):
        return (np.nan, np.nan, np.nan, np.nan, np.nan)
    code = int(code)
    block = (code - 1) // 18                 # 0..3 ⇒ condition
    cond = adult_cond_map[block]
    within = (code - 1) % 18                 # 0..17
    sex = 'male' if within < 9 else 'female'
    half = within % 9                        # 0..8
    educ_idx = half // 3                     # 0: 8th, 1: HS, 2: college
    race_idx = half % 3                      # 0: white, 1: black, 2: hispanic
    race = adult_race_map[race_idx]
    educ = adult_educ_map[educ_idx]
    age_child = np.nan                       # adults have no child age
    return (race, sex, educ, cond, age_child)

# ---------- CHILD VIGNETTES: CHLDVIG 14–16, 33–56 ----------

def decode_child_vignette(code):
    if pd.isna(code):
        return (np.nan, np.nan, np.nan, np.nan, np.nan)
    code = int(code)

    # 14–16: DEPRESSION, JOHN/MARY, BLACK, 8/14  (from codebook page)
    if 14 <= code <= 16:
        cond = 'depression_child'
        race = 'black'
        if code == 14:
            sex, age_child = 'male', 14
        elif code == 15:
            sex, age_child = 'female', 8
        else:  # 16
            sex, age_child = 'female', 14
        educ = np.nan
        return (race, sex, educ, cond, age_child)

    # 33–40, 41–48, 49–56: classic dep / impulsivity / perfectionism
    if 33 <= code <= 56:
        if 33 <= code <= 40:
            cond = 'classic_depression'
            base = 33
        elif 41 <= code <= 48:
            cond = 'impulsivity'
            base = 41
        else:
            cond = 'perfectionism'
            base = 49

        off = code - base          # 0..7
        race = 'white' if off < 4 else 'black'
        inner = off % 4            # 0..3 within race block
        sex  = 'male' if inner in (0, 1) else 'female'
        age_child = 8 if inner in (0, 2) else 14
        educ = np.nan
        return (race, sex, educ, cond, age_child)

    # anything else (asthma/normal/etc.) → treated as missing
    return (np.nan, np.nan, np.nan, np.nan, np.nan)

# ---------- APPLY TO EACH ROW ----------

def decode_row(row):
    if pd.notna(row['vigversn']) and row['vigversn'] < 73:
        return decode_adult_vignette(row['vigversn'])
    elif pd.notna(row['chldvig']):
        return decode_child_vignette(row['chldvig'])
    else:
        return (np.nan, np.nan, np.nan, np.nan, np.nan)

df[['vig_race', 'vig_sex', 'vig_educ', 'vig_cond', 'vig_child_age']] = (
    df.apply(decode_row, axis=1, result_type='expand')
)

In [ ]:
df[['vigversn', 'chldvig', 'vig_race', 'vig_sex', 'vig_educ', 'vig_cond', 'vig_child']].head(10)

In [ ]:
df[df['vig_child'] == 1][['vigversn', 'chldvig', 'vig_race', 'vig_sex', 'vig_educ', 'vig_cond', 'vig_child_age', 'vig_child']].sample(10, random_state=1)

In [ ]:
# Categorical vignette columns
vig_cat_cols = ['vig_race','vig_sex','vig_educ','vig_cond']

# Dummy for categorical attributes
X_vig_cat = pd.get_dummies(
    df[vig_cat_cols],
    prefix=['race','sex','educ','cond'],
    drop_first=True
)

# Dummy: child_age_14 = 1 if age 14, else 0
df['vig_age_14'] = (df['vig_child_age'] == 14).astype(int)

# Combine everything
X_vig = pd.concat([X_vig_cat], axis=1)

df = pd.concat([df, X_vig], axis=1)

In [ ]:
# Convert sex into dummy
df['female'] = (df['sex'] == 2).astype(int)

# Center age
df['age_c'] = df['age'] - df['age'].mean()

# Create dummies for respondent characteristics (DO NOT concat back to df)
df_degree = pd.get_dummies(df['degree'], prefix='deg', drop_first=True)
df_race   = pd.get_dummies(df['race'],   prefix='race', drop_first=True)
df_year   = pd.get_dummies(df['year'],   prefix='year', drop_first=True)
df_region = pd.get_dummies(df['region'], prefix='region', drop_first=True)

print(df_degree.head())

# Add them back to df (this is correct)
df = pd.concat(
    [df, df_degree, df_race, df_year, df_region],
    axis=1
)

print("Done adding respondent dummies. df now has:", df.shape[1], "columns.")

In [ ]:
# respondent controls
resp_controls = (
    ['female', 'age_c']
    + list(df_degree.columns)
    + list(df_race.columns)
    + list(df_year.columns)
    + list(df_region.columns)
)

# vignette controls (vignette dummies you created earlier in X_vig)
vig_controls = list(X_vig.columns) + ['vig_age_14']

# full set of regression controls
all_dummies_and_controls = resp_controls + ['vig_child'] + vig_controls

In [ ]:
df['sex_male'].value_counts()

In [ ]:
df[all_dummies_and_controls].columns

In [ ]:
df[all_dummies_and_controls].head()

In [ ]:
model = smf.ols(
    formula="""
        stigma_index
        ~ C(sex)
        + age
        + C(degree)
        + C(race)
        + C(year)
        + C(region)
        + C(vig_race)
        + C(vig_sex)
        + C(vig_educ)
        + C(vig_cond)
        + vig_child
        + C(vig_child_age)
    """,
    data=df
).fit(cov_type='HC3')

print(model.summary())

# Saving my Data 

In [ ]:
# Save my intermediate datasets

df_full.to_csv(path.join(intermediate_data_dir, "all_gss_w_vignettes.csv"), index=False)

df.to_csv(path.join(intermediate_data_dir, "gss_no_normal_vignettes.csv"), index=False)